<a href="https://colab.research.google.com/github/VOX304/SchoolChatbot/blob/main/Preprocessing/Colab_preprocessing/image_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install paddleocr ultralytics opencv-python numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
  

In [ ]:
!pip install pdf2image python-docx pdfplumber paddlepaddle-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 72.6 MB/s eta 0:00:00


In [ ]:
!pip install paddlepaddle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: opt-einsum
    Found existing installation: opt_einsum 3.4.0
    Uninstalling opt_einsum-3.4.0:
      Successfully uninstalled opt_einsum-3.4.0


In [ ]:
# Import required libraries
import cv2
import numpy as np
import json
import os
from paddleocr import PaddleOCR
from ultralytics import YOLO
import pdfplumber
from docx import Document
from PIL import Image

# Initialize models
ocr = PaddleOCR(use_angle_cls=True, lang='vi')  # Vietnamese OCR with angle correction
yolo = YOLO('yolov8s.pt')  # Load YOLOv8 small model

# Confidence thresholds
OCR_CONF_THRESH = 0.7  # Filter out OCR results below this
OBJ_CONF_THRESH = 0.5  # Filter out detected objects below this

def extract_images_from_pdf(pdf_path, output_folder="images"):
    """Extract actual images from a PDF file (excluding full pages)."""
    os.makedirs(output_folder, exist_ok=True)
    image_paths = []

    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages):
            for img_index, img in enumerate(page.images):
                img_data = pdf.extract_image(img["stream"])
                if img_data:
                    img_path = os.path.join(output_folder, f"pdf_img_{page_num}_{img_index}.jpg")
                    with open(img_path, "wb") as f:
                        f.write(img_data["image"])
                    image_paths.append(img_path)

    return image_paths

def extract_images_from_docx(docx_path, output_folder="images"):
    """Extract images from a DOCX file (more reliable method)."""
    os.makedirs(output_folder, exist_ok=True)
    image_paths = []
    doc = Document(docx_path)
    for i, rel in enumerate(doc.part.rels.values()):
        if "image" in rel.reltype:
            img_data = rel.target_part.blob
            img_path = os.path.join(output_folder, f"docx_img_{i}.jpg")
            with open(img_path, "wb") as f:
                f.write(img_data)
            image_paths.append(img_path)
    return image_paths

def detect_text(image_path):
    """Apply PaddleOCR to extract text from an image with confidence filtering."""
    results = ocr.ocr(image_path, cls=True)
    extracted_text = []
    for line in results:
        for word_info in line:
            text, confidence = word_info[1][0], word_info[1][1]
            if confidence >= OCR_CONF_THRESH:
                extracted_text.append(text)
    return extracted_text

def detect_objects(image_path):
    """Apply YOLOv8 to detect objects in an image with confidence filtering."""
    results = yolo(image_path)
    detected_objects = []
    for result in results:
        for box in result.boxes.data:
            conf = float(box[4])  # Confidence score
            if conf >= OBJ_CONF_THRESH:
                class_id = int(box[5])
                label = result.names[class_id]
                detected_objects.append(label)
    return detected_objects

def process_image(image_path):
    """Determine if an image contains text or objects and process accordingly."""
    text_results = detect_text(image_path)
    object_results = detect_objects(image_path)

    metadata = {
        "image_path": image_path,
        "extracted_text": text_results if text_results else None,
        "detected_objects": object_results if object_results else None
    }
    return metadata

def process_documents_in_folder(document_folder, output_json="image_metadata.json"):
    """Process all PDF and DOCX files in a folder and save metadata."""
    all_metadata = []
    os.makedirs(document_folder, exist_ok=True)

    for file_name in os.listdir(document_folder):
        file_path = os.path.join(document_folder, file_name)
        if file_name.endswith(".pdf"):
            image_paths = extract_images_from_pdf(file_path)
        elif file_name.endswith(".docx"):
            image_paths = extract_images_from_docx(file_path)
        else:
            continue  # Skip unsupported file types

        for img_path in image_paths:
            metadata = process_image(img_path)
            all_metadata.append(metadata)

    # Save metadata to JSON for easy VectorDB insertion
    with open(output_json, "w", encoding="utf-8") as f:
        json.dump(all_metadata, f, ensure_ascii=False, indent=4)

    return all_metadata

# Example usage
document_folder = "/content/sample_data/tcu"  # Replace with your folder path
metadata_results = process_documents_in_folder(document_folder)
print("Metadata saved to image_metadata.json")


[2025/03/21 08:27:37] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_l